In [27]:
import pandas as pd
import re

In [36]:
import time
import requests
from pathlib import Path
import json

In [37]:
from shapely.geometry import Point

### Форматирование адресов

Геокодирование адресов происходило через API геокодера OpenStreetMaps, который называется nominatim. 

Поисковый алгоритм nominatim плохо обрабатывает запросы, адреса в которых значительно отличаются от занесенных в его базу. Для взаимодействия с этим API следует форматировать адреса в установленном внутренними правилами OSM формате (https://wiki.openstreetmap.org/wiki/RU:%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F/%D0%A1%D0%BE%D0%B3%D0%BB%D0%B0%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BE%D0%B1_%D0%B8%D0%BC%D0%B5%D0%BD%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B8_%D0%B4%D0%BE%D1%80%D0%BE%D0%B3)

Низкая толерантность поиска к ошибкам привела к тому, что геокодирование проводилось в несколько этапов. При обнружении ошибок в данных алгоритм предобрабокт и запроса перерабатывался. Например, для случая поселков и городов в черте СПб следовало полностью удалить их статус "город" и "поселок"

In [28]:
def standartize_address(street):
    
    """
        Функция приводит основную часть адресов к стандартному формату. 
        Статусная часть (улица, бульвар и проч) приводится к полной форме.
        Если адрес состоит из прил + сущ, прилагательно выносится в начало. 
    """
    # обработка статусной части
    shorthands = {'ул.': 'улица', 
                  'пр.': 'проспект',
                  'ш.': 'шоссе',
                  'пер.': 'переулок',
                  'бульв.': 'бульвар',
                  'наб.': 'набережная'}
    
    street = re.sub('пос\.', '', street)
    street = re.sub('г\.', '', street)
    # запятой обычно отедяляются названия нас пунктов в черете города
    locality = None
    street = street.split(',')
    if len(street) == 2:
        locality = street[0]
        street = street[1]
    else:
        street = street[0]
    street = street.split(' ')
    if len(street) <= 1:
        return(street[0])
    else:
        status = street[-1]
        # обработка статусной части
        if status.lower() in shorthands:
            status = shorthands[status.lower()]
        # перенос прилагательного в составе названия
        if street[-2][-2:] in 'ая ий ый ое':
            if locality is not None:
                return ' '.join((locality + ',', ' '.join(street[:-1]), status))
            else:
                return ' '.join((' '.join(street[:-1]), status))
        # без переноса названия
        else:
            if locality is not None:
                return ' '.join((locality + ',', status, ' '.join(street[0:-2]), street[-2]))
            else:
                return ' '.join((status, ' '.join(street[0:-2]), street[-2]))

In [29]:
def postproc(line):
    """
        Некоторые адреса было удобнее обработать индивидуально.
    """
    post_proc = {'улица Лени Голикова': 'улица Лёни Голикова',
                    'набережная канала  Грибоедова': 'набережная канала Грибоедова',
                    'проспект Большой П.С.': 'Большой проспект П.С.',
                    'Пушкин,  Саперная улица': 'Пушкин, Сапёрная улица',
                    'Славы  Пр.': 'проспект Славы',
                    '2-ая Комсомольская улица': '2-я Комсомольская улица',
                    'Пискаревский проспект': 'Пискарёвский проспект',
                    'В.О. Большой пр.': 'Большой проспект В.О.',
                    'Б. Пушкарская улица': 'Большая Пушкарская улица',
                    'проспект  Королева': 'проспект Королёва',
                    'проспект Малый ВО': 'Малый проспект В.О.',
                    'улица  Замшина':'Замшина улица',
                    'улица Летчика Пилютова': 'улица Лётчика Пилютова',
                    'В.О. 14-я Линия': '14-я Линия В.О.',
                    # пос.Песочный,  Ленинградская улица: Песочный,  Ленинградская улица
                    'Мойки  наб.реки': 'набережная реки Мойки',
                    'проспект Малый П.С.': 'Малый проспект П.С.',
                    'проспект  Непокоренных': 'проспект Непокорённых',
                    'переулок  Басков': 'Басков переулок',
                    'улица Большая Зеленина': 'Большая Зеленина улица',
                    'Б. Монетная улица': 'Большая Монетная улица',
                    'Долгоозерная улица': 'Долгоозёрная улица',
                    'Чебышевская улица': 'Чебышёвская улица',
                    # пос. Металлострой,  Центральная улица: Металлострой,  Центральная улица
                    # г. Красное Село, Хвойный  пос.: Красное Село, Хвойный
                    'Парголово, Первого Мая улица': 'Парголово, улица Первого Мая',
                    '7-ая Красноармейская улица': '7-я Красноармейская улица',
                    # пос.Шушары,  Школьная улица: Шушары,  Школьная улица
                    # пос.Песочный, 16-й  квартал: Песочный, 16-й  квартал
                    'Ковалевская улица': 'Ковалёвская улица',
                    'набережная Черной речки': 'набережная Чёрной речки',
                    # пос.Шушары,  Пушкинская улица: Шушары, Пушкинская улица
                    # пос.Песочный, 6-й  квартал: Песочный, 6-й квартал
                    'улица  Панфилова': 'Панфилова улица',
                    # пос.Песочный,  Ленинградская улица: Песочный,  Ленинградская улица
                    # пос.Песочный, 10-й  квартал: Песочный, 10-й  квартал
                    'Б. Морская улица': 'Большая Морская улица',
                    'переулок  Поварской': 'Поварской переулок',
                    'улица  Опочинина': 'Опочинина улица',
                    # пос.Металлострой, Садовая  ул.: Металлострой, Садовая улица
                    # пос.Песочный, 9-й  квартал
                    # пос. Понтонный,  Южная улица
                    'Металлострой, Полевая  ул.': 'Металлострой, Полевая  улица',
                    'набережная Обводного кан.': 'набережная Обводного канала',
                    'улица  Полозова': 'Полозова улица',
                    'Пушкин, улица  Средняя': 'Пушкин, Средняя улица',
                    'улица  Подрезова': 'Подрезова улица',
                    'улица Бармалеева': 'Бармалеева улица',
                    'Б.Конюшенная улица': 'Большая Конюшенная улица',
                    'улица  Карташихина': 'Карташихина улица',
                    'Б.Московская улица': 'Большая Московская улица',
                    'переулок  Дровяной': 'Дровяной переулок',
                    '15-я линия,  В.О.': '15-я линия В.О.',
                    'Металлострой, Бойчука  ул.': 'Металлострой, Бойчука  улица',
                    'Пряжки Наб. р.': 'набережная реки Пряжки',
                    'Павловск,  Березовая улица': 'Павловск, Берёзовая улица',
                    'Песочный, Военный родок': 'Песочный, Военный городок',
                    'переулок  Свечной': 'Свечной переулок',
                    'улица  Крюкова': 'Крюкова улица',
                    'проспект  Лесной': 'Лесной проспект',
                    'улица  Шамшева': 'Шамшева улица',
                    'Б.Морская улица': 'Большая Морская улица',
                    'набережная р. Карповки': 'набережная реки Карповки'}
    return post_proc.get(line, line)

In [35]:
def full_address(locality, number, building):
    """
        Собираем полные адреса. 
        Литеры опускаются, т.к. незначительно влияют на точность позиционирования,
        а в датасете по умолчанию используется литера "А".
    """
    if building == 'nan':
        return 'Санкт-Петербург, ' + locality + ' ' + number
    else:
        return 'Санкт-Петербург, ' + locality + ' ' + number + ' к' + building

In [41]:
# загружаем информацию об адресе
data = pd.read_csv('data/housing_data.csv')
data = data[['addr_street', 'addr_number', 'addr_building', 'addr_district']]

In [33]:
# приводим форматирование адресов к требуемому osm
data['osm_street'] = data['addr_street'].map(standartize_address)
# авторы датасета придеживались единого формата не для всех адресов, заменим "неформатные" адреса подстановкой
data['osm_street'] = data['osm_street'].apply(lambda x: postproc(x))
# сконтруируем полный адрес
data['osm_address'] = data.apply(lambda x: full_address(x['osm_street'], x['addr_number'], str(x['addr_building'])), axis=1)

In [34]:
data.head()

,addr_street,addr_number,addr_building,osm_street,osm_address
0,Королева пр.,31,1,проспект Королёва,"Санкт-Петербург, проспект Королёва 31 к1"
1,Дачный пр.,33,1,Дачный проспект,"Санкт-Петербург, Дачный проспект 33 к1"
2,Будапештская ул.,34,NaN,Будапештская улица,"Санкт-Петербург, Будапештская улица 34"
3,Бухарестская ул.,86,2,Бухарестская улица,"Санкт-Петербург, Бухарестская улица 86 к2"
4,"г.Пушкин, Красносельское ш.",65,NaN,"Пушкин, Красносельское шоссе","Санкт-Петербург, Пушкин, Красносельское шоссе 65"


### Геокодинг

Для получения расположения адресов, делаем get-запрос к апи. Сохраняем полученный ответ для последующей работы. Для некоторых запросов ответ возращает несколько объектов. Берем из них только те, для которых район расположения объекта совпадает для ответа API и данных датасета. В противном случае, часты случаи, когда одноименные улицы разных населенных пунктов смешиваются. По возможности, берем координаты объекта типа building.

In [38]:
def geocode(df, path_to_dir):
    """
        Query nominatim api for address coordinates.
        Df should have a column named 'addr' formated
        so that nominatim understands it.
        Saves json files.
    """
    url = 'https://nominatim.openstreetmap.org/search'
    for index, row in df.iterrows():
        time.sleep(1)
        try:
            print(index, row['addr'])
            params = {'q': row['addr'], 'format':'json'}
            g = requests.get(url, params=params)
            g = g.json()
            print(g)
            with open(Path(path_to_dir, str(index), 'w')) as f:
                json.dump(g, f)
        except Exception as err:
            print(err)

In [40]:
def extract_coordinates(df):
    """
        Get coordinates from saved api responses.
        District name from the dataset must be present in the address
        returned by the api. It guarantees that streets with identical names
        are not confused.
    """
    df_addr = pd.DataFrame(columns=['index', 'address', 'class', 'geometry'])
    for index, row in error.iterrows():
        with open(f'./jsons/{index}', 'r') as f:
            response = json.load(f)
            for item in response:
                candidate = None
                if (row['addr_district'] in item['display_name']):
                    if not candidate:
                        candidate = {'index': index, 'address': item['display_name'],
                                     'class': item['class'], 'geometry': Point(item['lat'], item['lon'])}
                    if (item['class'] == 'building'):
                        candidate = {'index': index, 'address': item['display_name'],
                                     'class': item['class'], 'geometry': Point(item['lat'], item['lon'])}
                        break
            if candidate:
                df_addr = df_addr.append(candidate, ignore_index=True)
    return df_addr

In [ ]:
geocode(rest, './jsons')
points = extract_coordinates(rest)

### Форматирование ответов и сохранение

In [ ]:
# epsg:4326 -- система координат, используемая OSM
points = gpd.GeoDataFrame(points, geometry='geometry', crs={'init': 'epsg:4326'})
# чтобы не терять индекс при сохранении
points['idx'] = points.index
points[['idx', 'geometry']].dropna().to_file('shapes/points/points.shp', encoding='utf-8')